In [3]:
from random import randrange
import numpy as np
from math import log

class DecisionTreeClassifier(object):
    
    # Initailizing values
    def __init__(self, max_depth):
        
        self.depth = 0
        self.max_depth = max_depth
    
    
    # Evaluate an algorithm using a train/test split
    def train_test_split(self,X, y, split):

        X_train = list()
        y_train = list()
        train_size = split * len(X)
        X_test = list(X)
        y_test = list(y)
        
        while len(X_train) < train_size:
            index = randrange(len(X_test))
            X_train.append(X_test.pop(index))
            y_train.append(y_test.pop(index))
        
        return X_train,X_test,y_train,y_test
    
    # Calculation of entropy
    def entropy_func(self, c, n):
    
        return -(c*1.0/n)*log(c*1.0/n, 2)

    
    def entropy_cal(self, c1, c2):
    
        if c1== 0 or c2 == 0:  # when there is only one class in the group, entropy is 0
            return 0
        return self.entropy_func(c1, c1+c2) + self.entropy_func(c2, c1+c2)


    # get the entropy of one big circle showing above
    def entropy_of_one_division(self, division): 
    
        s = 0
        n = len(division)
        classes = set(division)
        
        for c in classes:   # for each class, get entropy
            n_c = sum(division==c)
            e = n_c * 1.0/n * self.entropy_cal(sum(division==c), sum(division!=c)) # weighted avg
            s += e
        return s, n

    
    # The whole entropy of two big circles combined
    def get_entropy(self, y_predict, y_real):
    
        if len(y_predict) != len(y_real):
            print('They have to be the same length')
            return None
        
        n = len(y_real)

        s_true, n_true = self.entropy_of_one_division(y_real[y_predict]) # left hand side entropy
        s_false, n_false = self.entropy_of_one_division(y_real[~y_predict]) # right hand side entropy
        s = n_true*1.0/n * s_true + n_false*1.0/n * s_false # overall entropy, again weighted average
        
        return s
    
    # Fitting the model
    def fit(self, x, y, par_node={}, depth=0):
        
        x = np.array(x)
        y = np.array(y)
        
        if par_node is None: 
            return None
        elif len(y) == 0:
            return None
        elif self.all_same(y):
            return {'val':y[0]}
        elif depth >= self.max_depth:
            return None
        else: 
            col, cutoff, entropy = self.find_best_split_of_all(x, y)    # find one split given an information gain 
            y_left = y[x[:, col] < cutoff]
            y_right = y[x[:, col] >= cutoff]
            
            par_node = {'col': iris.feature_names[col], 'index_col':col,
                        'cutoff':cutoff,
                       'val': np.round(np.mean(y))}
            
            par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)
            par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)
            
            self.depth += 1 
            self.trees = par_node
            return par_node
    
    
    def find_best_split_of_all(self, x, y):
        col = None
        min_entropy = 1
        cutoff = None
        
        for i, c in enumerate(x.T):
            entropy, cur_cutoff = self.find_best_split(c, y)
            
            if entropy == 0:    # find the first perfect cutoff. Stop Iterating
                return i, cur_cutoff, entropy
            elif entropy <= min_entropy:
                min_entropy = entropy
                col = i
                cutoff = cur_cutoff
        
        return col, cutoff, min_entropy
    
    
    # Find the best split
    def find_best_split(self, col, y):
        min_entropy = 10
        n = len(y)
        
        for value in set(col):
            y_predict = col < value
            my_entropy = self.get_entropy(y_predict, y)
            
            if my_entropy <= min_entropy:
                min_entropy = my_entropy
                cutoff = value
        
        return min_entropy, cutoff
    
    
    def all_same(self, items):
        return all(x == items[0] for x in items)
                                           
    # Prediction function
    def predict(self, x):
        tree = self.trees
        results = np.array([0]*len(x))
        
        for i, c in enumerate(x):
            results[i] = self.get_prediction(c)
        
        return results
    
    # Predicting values
    def get_prediction(self, row):
        cur_layer = self.trees
        
        while cur_layer.get('cutoff'):
            if row[cur_layer['index_col']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val')
        
    # Calculate score
    def score(self,X_test, y_test):
        correct = 0
        predicted = list()
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        
        for row in X_test:
            output = self.predict([row])
            predicted.append(output)
        
        for i in range(len(y_test)):
            if y_test[i] == predicted[i]:
                correct = correct + 1
        
        return correct / float(len(y_test))
    
        
if __name__ == "__main__":
    
    from sklearn.datasets import load_iris
    iris = load_iris()
    X = iris.data
    y = iris.target

    model = DecisionTreeClassifier(5)
    X_train, X_test, y_train, y_test = model.train_test_split(X,y,0.7)
    
    model.fit(X_train, y_train)
    print(model.predict([[5.1, 3.8, 1.6, 0.2]]))
    print(model.score(X_test,y_test))

[0]
0.9777777777777777
